In [1]:
from collections import defaultdict
import time
import random
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

CUDA = torch.cuda.is_available()
print("CUDA: %s" % CUDA)

# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]

def read_dataset(filename):
    with open(filename, "r") as f:
        for line in f:
            tag, words = line.lower().strip().split(" ||| ")
            yield ([w2i[x] for x in words.split(" ")], t2i[tag])


#Baca dataset
train = list(read_dataset("dataset/classes/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("dataset/classes/dev.txt"))
nwords = len(w2i)
ntags = len(t2i)

In [ ]:
def preprocess(batch):
    """ Add zero-padding to a batch. """

    tags = [example.tag for example in batch]

    # add zero-padding to make all sequences equally long
    seqs = [example.words for example in batch]
    max_length = max(map(len, seqs))
    seqs = [seq + [PAD] * (max_length - len(seq)) for seq in seqs]

    return seqs, tags

In [2]:
class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, output_dim):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, output_dim)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        bow = torch.sum(embeds, 1)
        logits = self.linear(bow)
        return logits

In [3]:
model = CBOW(nwords, 64, ntags)

#Pindahkan model ke GPU apabila tersedia
if CUDA:
    model.cuda()
    
print(model)

CBOW(
  (embeddings): Embedding(17614, 64)
  (linear): Linear(in_features=64, out_features=5, bias=True)
)


In [4]:
def evaluate(model, data):
    """Evaluate a model on a data set."""
    correct = 0.0
    
    for words, tag in data:
        lookup_tensor = Variable(torch.LongTensor([words]))
        scores = model(lookup_tensor)
        predict = scores.data.numpy().argmax(axis=1)[0]

        if predict == tag:
            correct += 1

    return correct, len(data), correct/len(data)

In [5]:
optimizer = optim.SGD(model.parameters(), lr=0.001)

for ITER in range(100):

    random.shuffle(train)
    train_loss = 0.0
    start = time.time()

    for words, tag in train:

        # forward pass
        lookup_tensor = Variable(torch.LongTensor([words]))
        scores = model(lookup_tensor)
        loss = nn.CrossEntropyLoss()
        target = Variable(torch.LongTensor([tag]))
        output = loss(scores, target)
        train_loss += output.data[0]

        # backward pass
        model.zero_grad()
        output.backward()

        # update weights
        optimizer.step()

    print("iter %r: train loss/sent=%.4f, time=%.2fs" % 
          (ITER, train_loss/len(train), time.time()-start))

    # evaluate
    _, _, acc = evaluate(model, dev)
    print("iter %r: test acc=%.4f" % (ITER, acc))

/home/lintangsutawika/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


iter 0: train loss/sent=1.9944, time=21.80s
iter 0: test acc=0.2443
iter 1: train loss/sent=1.9255, time=27.85s
iter 1: test acc=0.2443
iter 2: train loss/sent=1.9225, time=24.28s
iter 2: test acc=0.2434
iter 3: train loss/sent=1.9228, time=21.51s
iter 3: test acc=0.2325
iter 4: train loss/sent=1.9004, time=20.36s
iter 4: test acc=0.2507
iter 5: train loss/sent=1.9077, time=21.20s
iter 5: test acc=0.2670
iter 6: train loss/sent=1.9013, time=18.74s
iter 6: test acc=0.2652
iter 7: train loss/sent=1.8970, time=18.38s
iter 7: test acc=0.2470
iter 8: train loss/sent=1.8788, time=20.04s
iter 8: test acc=0.2652
iter 9: train loss/sent=1.8940, time=20.37s
iter 9: test acc=0.2534
iter 10: train loss/sent=1.8842, time=19.17s
iter 10: test acc=0.2443
iter 11: train loss/sent=1.8684, time=21.49s
iter 11: test acc=0.2879
iter 12: train loss/sent=1.8772, time=19.58s
iter 12: test acc=0.2552
iter 13: train loss/sent=1.8736, time=19.57s
iter 13: test acc=0.2779
iter 14: train loss/sent=1.8709, time=19

KeyboardInterrupt: 

Deeper CBOW
Tambahkan hidden layer, coba bandingkan.

In [ ]:
class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, output_dim):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, output_dim)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        bow = torch.sum(embeds, 1)
        logits = self.linear(bow)
        return logits